In [2]:
import sys
sys.path.insert(0, '../')
from humannotator import Annotator, task_factory, load_data
import pandas as pd

In [3]:
df = pd.read_csv('news.csv', index_col=0)
data = load_data(df, items_cols=['title', 'date'], id_col='news_id')

In [4]:
instruction = "Choose one of the following options:"
choices={
    '0': 'not adverse media',
    '1': 'adverse media',
    '3': 'exclude from dataset'
}
task1 = task_factory(choices, 'Choice', instruction=instruction)
task2 = task_factory('bool', 'Relevant', instruction="State if the article is relevant") 

annotator = Annotator(df, [task1, task2])
annotator = Annotator(data, [task1, task2])
# annotator = Annotator(data, annotations, text_display=True)

In [ ]:
annotator(data.ids)

In [6]:
annotator.annotations.data

,Choice,Relevant,timestamp
052632_2015-02-28,not adverse media,False,2019-09-01 21:43:32.255558
071607_2016-12-12,not adverse media,False,2019-09-01 21:43:33.207091
141694_2016-02-10,not adverse media,False,2019-09-01 21:43:34.076486
137157_2017-02-09,not adverse media,False,2019-09-01 21:43:34.925714
034187_2016-09-27,not adverse media,False,2019-09-01 21:43:35.788009
018678_2017-04-23,not adverse media,False,2019-09-01 21:43:36.646894
120386_2016-11-14,not adverse media,False,2019-09-01 21:43:37.525339
135236_2016-11-10,not adverse media,False,2019-09-01 21:43:38.440891
184514_2017-03-17,not adverse media,False,2019-09-01 21:43:39.612662
106098_2017-06-02,not adverse media,False,2019-09-01 21:43:41.020445
